In [3]:
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    TRAIN_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    TRANSFORMED_DATA_PATH,
    N_COMPONENTS,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    EPSILON,
    RECOMMENDATIONS_KEY_CB,
    RECOMMENDATIONS_PATH,
)

In [20]:
# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

In [4]:
test_data = {"ep1": [2,3,4],
             "ep2": [8,4,6]}
test_df = pd.DataFrame(test_data)
# Transpose and reset index
reshaped_df = test_df.T.reset_index()

# Rename columns
reshaped_df.columns = ['episode', 'feature1', 'feature2', 'feature3']
reshaped_dict = reshaped_df.to_dict(orient="list")
reshaped_dict

{'episode': ['ep1', 'ep2'],
 'feature1': [2, 8],
 'feature2': [3, 4],
 'feature3': [4, 6]}

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

text = "Lyssky med milliarder i Dubai"
embedding = model.encode(text)

print(embedding.shape)  # e.g. (384,)


ModuleNotFoundError: No module named 'sentence_transformers'